# BCRP
## 2024 University Extension Course - High Dimensional Econometrics
### Final Exam - Coding

**Instructions**: Please write down your code and _execute_ it printing the relevant output before submitting this ```ipynb``` file. Full marks will _not_ be given only for correct answers but for code elegance as well as _elegant_ display of the results. Donwload this notebook with your _executed_ answers as ```XXXX.ipynb``` where ```XXX``` is your student code and mail it to manuel.vasquez@bcrp.gob.pe and luciana.loayza@bcrp.gob.pe.

In [2]:
import stata_setup, os
if os.name == 'nt':
    stata_setup.config('C:/Program Files/Stata17/','mp')
else:
    stata_setup.config('/usr/local/stata17','mp')


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 4-core  perpetual
Serial number: 501706303466
  Licensed to: David Tomas Jacho-Chavez
               Emory University

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000 but can be increased;
          see help set_maxvar.


1. Take the ```cpsmar09``` dataset and the subsample of Asian women $(n=1149)$. Estimate a Lasso linear regression of $\log$ (wage) on the following variables: _education_; dummies for _education_ equalling $12,13,14,15,16,18$, and 20; _experience_/40 in powers from 1 to 9 ; dummies for marriage categories married, divorced, separated, widowed, never married; dummies for the four regions; dummy for union membership. Report the estimated model and coefficients. **Note**: You are welcome to use ```Stata``` default plugin value of the needed hyperparameter with a random seed set to 2024. [20 points]

In [3]:
import pandas as pd
import numpy as np
cps09mar = pd.read_stata('../Data/cps09mar.dta')

In [4]:
# We generate the transformation of variables
cps09mar['wage'] = np.log(cps09mar.earnings/(cps09mar.hours*cps09mar.week))
cps09mar['experience'] = cps09mar.age - cps09mar.education - 6

In [5]:
%%stata -d cps09mar -force
forvalues i=12(1)20 {
	gen educ_`i' = 1 if education==`i'
    replace educ_`i' = 0 if educ_`i'==.
}
drop educ_17 educ_19

rename educ_12 High_School_Graduate
rename educ_13 College
rename educ_14 Associate_degree
rename educ_16 Bachelors_degree
rename educ_18 Masters_degree
rename educ_20 Doctorate_degree


. forvalues i=12(1)20 {
  2.         gen educ_`i' = 1 if education==`i'
  3.     replace educ_`i' = 0 if educ_`i'==.
  4. }
(36,846 missing values generated)
(36,846 real changes made)
(41,488 missing values generated)
(41,488 real changes made)
(45,134 missing values generated)
(45,134 real changes made)
(50,742 missing values generated)
(50,742 real changes made)
(39,102 missing values generated)
(39,102 real changes made)
(50,742 missing values generated)
(50,742 real changes made)
(46,072 missing values generated)
(46,072 real changes made)
(50,742 missing values generated)
(50,742 real changes made)
(48,867 missing values generated)
(48,867 real changes made)

. drop educ_17 educ_19

. 
. rename educ_12 High_School_Graduate

. rename educ_13 College

. rename educ_14 Associate_degree

. rename educ_16 Bachelors_degree

. rename educ_18 Masters_degree

. rename educ_20 Doctorate_degree

. 


In [6]:
%%stata
forvalues i=1(1)9 {
	gen exp_`i' = (experience/40)^(`i')
}


. forvalues i=1(1)9 {
  2.         gen exp_`i' = (experience/40)^(`i')
  3. }

. 


In [7]:
%%stata

gen mar_1= 1 if (marital==1 | marital==2 | marital==3)
replace mar_1 = 0 if mar_1==. 
forvalues i=4(1)7 {
	generate mar_`i' = 1 if (marital==`i')
    replace mar_`i' = 0 if mar_`i'==.
}

rename mar_1 Married
rename mar_4 Widowed
rename mar_5 Divorced
rename mar_6 Separated
rename mar_7 Never_married


. 
. gen mar_1= 1 if (marital==1 | marital==2 | marital==3)
(17,032 missing values generated)

. replace mar_1 = 0 if mar_1==. 
(17,032 real changes made)

. forvalues i=4(1)7 {
  2.         generate mar_`i' = 1 if (marital==`i')
  3.     replace mar_`i' = 0 if mar_`i'==.
  4. }
(50,030 missing values generated)
(50,030 real changes made)
(45,290 missing values generated)
(45,290 real changes made)
(49,667 missing values generated)
(49,667 real changes made)
(40,949 missing values generated)
(40,949 real changes made)

. 
. rename mar_1 Married

. rename mar_4 Widowed

. rename mar_5 Divorced

. rename mar_6 Separated

. rename mar_7 Never_married

. 


In [8]:
%%stata
forvalues i=1(1)4 {
	generate region_`i' = 1 if region==`i'
    replace region_`i' = 0 if region_`i'==.
}

rename region_1 Northeast
rename region_2 Midwest
rename region_3 South
rename region_4 West


. forvalues i=1(1)4 {
  2.         generate region_`i' = 1 if region==`i'
  3.     replace region_`i' = 0 if region_`i'==.
  4. }
(41,043 missing values generated)
(41,043 real changes made)
(38,717 missing values generated)
(38,717 real changes made)
(34,658 missing values generated)
(34,658 real changes made)
(37,808 missing values generated)
(37,808 real changes made)

. 
. rename region_1 Northeast

. rename region_2 Midwest

. rename region_3 South

. rename region_4 West

. 


In [9]:
%%stata
global region Midwest South West
global educ High_School_Graduate College Associate_degree educ_15 Bachelors_degree Masters_degree Doctorate_degree 
global marriage Widowed Divorced Separated Never_married
global exper exp_1 exp_2 exp_3 exp_4 exp_5 exp_6 exp_7 exp_8 exp_9

# Solo las variables de matrimonio y de región requieren eliminar una categoría base.


. global region Midwest South West

. global educ High_School_Graduate College Associate_degree educ_15 Bachelors_d
> egree Masters_degree Doctorate_degree 

. global marriage Widowed Divorced Separated Never_married

. global exper exp_1 exp_2 exp_3 exp_4 exp_5 exp_6 exp_7 exp_8 exp_9

. 
. # Solo las variables de matrimonio y de región requieren eliminar una categor
> ía base.
Unknown #command
. 


In [10]:
%stata lasso linear wage i.education $educ $exper $marriage $region union if (race==4 & female==1), folds(20) rseed(52) nolog
%stata estimates store lasso
%stata ereturn display


note: convergence for the lasso penalty = 0.000192 not reached after 100000
      iterations; solutions for larger penalty values returned.

Lasso linear model                          No. of obs        =      1,149
                                            No. of covariates =         30
Selection: Cross-validation                 No. of CV folds   =         20

--------------------------------------------------------------------------
         |                                No. of      Out-of-      CV mean
         |                               nonzero       sample   prediction
      ID |     Description      lambda     coef.    R-squared        error
---------+----------------------------------------------------------------
       1 |    first lambda    .2064107         0      -0.0017     .4615709
      71 |   lambda before    .0003065        24       0.2644     .3389903
    * 72 | selected lambda    .0002793        24       0.2645     .3389424
      73 |    lambda after    .0

In [11]:
%%stata -eret eret1
estimates store lasso

In [12]:
allvars_sel = eret1['e(allvars_sel)'].split()
b = eret1['e(b)']

allvars_sel.append('constant')

if isinstance(b, np.ndarray):
    b = b.tolist()
if len(b) == 1 and isinstance(b[0], list):
    b = b[0]

In [13]:
df = pd.DataFrame({'Covariates': allvars_sel, 'Penalized unstandardized coefficients': b})

html = df.to_html(index=False)

from IPython.display import display, HTML
display(HTML(html))

2. Repeat the above exercise using the subsample of Hispanic men $(n=4547)$. **Note**: You are welcome to use ```Stata``` default plugin value of the needed hyperparameter with a random seed set to 2024. [10 points]

In [14]:
%stata lasso linear wage i.education $educ $exper $marriage $region union if (hisp==1 & female==0), folds(20) rseed(52) nolog
%stata estimates store lasso
%stata ereturn display

note: convergence for the lasso penalty = 0.000027 not reached after 100000
      iterations; solutions for larger penalty values returned.

Lasso linear model                          No. of obs        =      4,547
                                            No. of covariates =         30
Selection: Cross-validation                 No. of CV folds   =         20

--------------------------------------------------------------------------
         |                                No. of      Out-of-      CV mean
         |                               nonzero       sample   prediction
      ID |     Description      lambda     coef.    R-squared        error
---------+----------------------------------------------------------------
       1 |    first lambda    .1429747         0      -0.0001       .43012
      88 |   lambda before    .0000437        25       0.2783     .3103735
    * 89 | selected lambda    .0000398        25       0.2784     .3103639
      90 |    lambda after    .00

In [15]:
%%stata -eret eret2
estimates store lasso

In [16]:
allvars_sel = eret2['e(allvars_sel)'].split()
b = eret2['e(b)']

allvars_sel.append('constant')

if isinstance(b, np.ndarray):
    b = b.tolist()
if len(b) == 1 and isinstance(b[0], list):
    b = b[0]

In [17]:
df = pd.DataFrame({'Covariates': allvars_sel, 'Penalized unstandardized coefficients': b})

html = df.to_html(index=False)

from IPython.display import display, HTML
display(HTML(html))

3. You will replicate and extend the work reported in the Chapter of Bruce Hansen's textbook relating to Card (1995). The data is from the author's website, and is posted as ```Card1995```. The model we focus on is labeled 2SLS(a) in Table 12.1 in Chapter 12, which uses _public_ and _private_ as instruments for _edu_. The variables you will need for this exercise include _lwage76_, _ed76_, _age76_, _smsa76r_, _reg76r_, _nearc2_, _nearc4_, _nearc4a_, _nearc4b_. See the description file for definitions.
$$
\log (\text { wage })=\beta_0+\beta_1 e d u+\beta_2 \exp +\beta_3 \exp ^2 / 100+\beta_4 \operatorname{south}+\beta_5 \text { Black }+e
$$
where _edu=education_ (Years), _exp=experience_ (Years), and _south_ and _Black_ are regional and racial dummy variables. The variables $\exp =a g e-e d u-6$ and $\exp ^2 / 100$ are not in the dataset, they need to be generated.

(a) First, replicate the reduced form regression presented in the final column of Table 12.2 in Bruce Hansen's relevant Chapter, and the 2SLS regression described above (using _public_ and _private_ as instruments for _edu_) to verify that you have the same variable defintions. [3 point]

In [18]:
from pystata import stata

In [19]:
stata.run('''
* Clear memory and load the data
use ../Data/Card1995, clear
''',quietly=True)

In [20]:
%%stata
set more off
gen exp = age76 - ed76 -6
gen exp2 = (exp^2)/100
* Dropping observations with missing wage
drop if lwage76==.


. set more off

. gen exp = age76 - ed76 -6

. gen exp2 = (exp^2)/100

. * Dropping observations with missing wage
. drop if lwage76==.
(603 observations deleted)

. 


In [21]:
stata.run('''
reg ed76 exp exp2 black reg76r smsa76r nearc4a nearc4b, r
estimates store Pub_priv
ivregress 2sls lwage76 exp exp2 black reg76r smsa76r (ed76=nearc4a nearc4b), r
''')


. 
. reg ed76 exp exp2 black reg76r smsa76r nearc4a nearc4b, r

Linear regression                               Number of obs     =      3,010
                                                F(7, 3002)        =     525.76
                                                Prob > F          =     0.0000
                                                R-squared         =     0.4762
                                                Root MSE          =     1.9396

------------------------------------------------------------------------------
             |               Robust
        ed76 | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
         exp |  -.4132653   .0320273   -12.90   0.000    -.4760631   -.3504675
        exp2 |   .0927963    .170775     0.54   0.587    -.2420515    .4276441
       black |  -1.006308   .0879844   -11.44   0.000    -1.178824   -.8337924
      reg76r |   -.267108   .

(b) Now try a different reduced form model. The variable _nearc2_ means "grew up near a 2-year college". See if adding it to the reduced form equation is useful. [5 points]

In [22]:
stata.run('''
reg ed76 exp exp2 black reg76r smsa76r nearc4a nearc4b nearc2, robust
estimates store With_2yr_coll
''')


. 
. reg ed76 exp exp2 black reg76r smsa76r nearc4a nearc4b nearc2, robust

Linear regression                               Number of obs     =      3,010
                                                F(8, 3001)        =     460.20
                                                Prob > F          =     0.0000
                                                R-squared         =     0.4764
                                                Root MSE          =     1.9396

------------------------------------------------------------------------------
             |               Robust
        ed76 | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
         exp |  -.4127948   .0320079   -12.90   0.000    -.4755545   -.3500351
        exp2 |   .0894569    .170675     0.52   0.600     -.245195    .4241088
       black |  -1.010604   .0879567   -11.49   0.000    -1.183065   -.8381425
      reg76r |  -

(c) Now try more interactions in the reduced form. Create the interactions $nearc4a*age76$ and $nearc4a*age76^2 / 100$, and add them to the reduced form equation. Estimate this by least squares. Interpret the coefficients on the two new variables. [10 points]

In [23]:
stata.run('''
gen int_1 = nearc4a*age76
gen int_2 = (nearc4a*age76^2)/100
''')


. 
. gen int_1 = nearc4a*age76

. gen int_2 = (nearc4a*age76^2)/100

. 


In [24]:
stata.run('''
reg ed76 exp exp2 black reg76r smsa76r nearc4a nearc4b int_1 int_2 , robust
estimates store With_int
''')


. 
. reg ed76 exp exp2 black reg76r smsa76r nearc4a nearc4b int_1 int_2 , robust

Linear regression                               Number of obs     =      3,010
                                                F(9, 3000)        =     545.44
                                                Prob > F          =     0.0000
                                                R-squared         =     0.6505
                                                Root MSE          =     1.5849

------------------------------------------------------------------------------
             |               Robust
        ed76 | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
         exp |  -.6717638    .031643   -21.23   0.000     -.733808   -.6097196
        exp2 |   .5537512   .1688994     3.28   0.001     .2225808    .8849217
       black |  -.6877297   .0751638    -9.15   0.000    -.8351076   -.5403519
      reg76

First, the constant of the first interaction is 1.48, which indicates that, when a person is close to a private college, the impact of the age in their education is greater. Perhaps it is because of the incentive to go to private college when a person grows up. Second, int_2 shows that this effect of living near to a private college with age may become weaker and even negative from an specific age. That means that the negative coefficient shows a cuadratic and concave behavior.

(d) Estimate the structural equation by 2SLS using the expanded instrument set {$nearc4a$, $nearc4b$, $nearc4a*age76$, $nearc4a*age76^2 / 100$}. What is the impact on the structural estimate of the return to schooling? [6 points]

In [25]:
stata.run('''
ivregress 2sls lwage76 exp exp2 black reg76r smsa76r (ed76=nearc4a nearc4b int_1 int_2), r
estimates store With_2sls
''')


. 
. ivregress 2sls lwage76 exp exp2 black reg76r smsa76r (ed76=nearc4a nearc4b in
> t_1 int_2), r

Instrumental variables 2SLS regression            Number of obs   =      3,010
                                                  Wald chi2(6)    =    1018.86
                                                  Prob > chi2     =     0.0000
                                                  R-squared       =     0.2891
                                                  Root MSE        =     .37412

------------------------------------------------------------------------------
             |               Robust
     lwage76 | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
        ed76 |   .0825386   .0062178    13.27   0.000      .070352    .0947252
         exp |    .087094   .0070498    12.35   0.000     .0732766    .1009115
        exp2 |  -.2247205   .0319734    -7.03   0.000    -.2873872   -.1

Each year of education increases in 8.25% the wages.

4. Take the ```cpsmar09``` dataset and the subsample of white male Hispanics. Estimate the regression

$$
\widehat{\log (\text { wage })}=\beta_1 \text { education }+\beta_2 \text { experience }+\beta_3 \text { experience }^2 / 100+\beta_4 \text {. }
$$

(a) Report the coefficient estimates and robust standard errors. [8 points]

In [26]:
import pandas as pd
import numpy as np
cps09mr = pd.read_stata('../Data/cps09mar.dta')
cps09mr.head()

age  female  hisp  education  earnings  hours  week  union  uncov  region  \
0  52.0     0.0   0.0       12.0  146000.0   45.0  52.0    0.0    0.0     1.0   
1  38.0     0.0   0.0       18.0   50000.0   45.0  52.0    0.0    0.0     1.0   
2  38.0     0.0   0.0       14.0   32000.0   40.0  51.0    0.0    0.0     1.0   
3  41.0     1.0   0.0       13.0   47000.0   40.0  52.0    0.0    0.0     1.0   
4  42.0     0.0   0.0       13.0  161525.0   50.0  52.0    1.0    0.0     1.0   

   race  marital  
0   1.0      1.0  
1   1.0      1.0  
2   1.0      1.0  
3   1.0      1.0  
4   1.0      1.0

In [27]:
cps09mr['wage'] = np.log(cps09mr.earnings/(cps09mr.hours*cps09mr.week))
cps09mr['experience'] = cps09mr.age - cps09mr.education - 6
cps09mr['exp2'] = (cps09mr.experience**2)/100
cps09mr['mbnt'] = np.logical_and(np.logical_and(cps09mr.hisp==1, cps09mr.race==1), cps09mr.female!=1)

In [28]:
cps09mr.head()  #-->para ver que se han añadido correctamente las nuevas variables a la data

age  female  hisp  education  earnings  hours  week  union  uncov  region  \
0  52.0     0.0   0.0       12.0  146000.0   45.0  52.0    0.0    0.0     1.0   
1  38.0     0.0   0.0       18.0   50000.0   45.0  52.0    0.0    0.0     1.0   
2  38.0     0.0   0.0       14.0   32000.0   40.0  51.0    0.0    0.0     1.0   
3  41.0     1.0   0.0       13.0   47000.0   40.0  52.0    0.0    0.0     1.0   
4  42.0     0.0   0.0       13.0  161525.0   50.0  52.0    1.0    0.0     1.0   

   race  marital      wage  experience   exp2   mbnt  
0   1.0      1.0  4.133456        34.0  11.56  False  
1   1.0      1.0  3.061872        14.0   1.96  False  
2   1.0      1.0  2.752786        18.0   3.24  False  
3   1.0      1.0  3.117780        22.0   4.84  False  
4   1.0      1.0  4.129148        23.0   5.29  False

In [29]:
%%stata -d cps09mr -force
regress wage education experience exp2 if mbnt==1
hettest


. regress wage education experience exp2 if mbnt==1

      Source |       SS           df       MS      Number of obs   =     4,230
-------------+----------------------------------   F(3, 4226)      =    428.77
       Model |  423.653968         3  141.217989   Prob > F        =    0.0000
    Residual |  1391.87252     4,226  .329359328   R-squared       =    0.2334
-------------+----------------------------------   Adj R-squared   =    0.2328
       Total |  1815.52649     4,229   .42930397   Root MSE        =     .5739

------------------------------------------------------------------------------
        wage | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
   education |    .090449   .0027367    33.05   0.000     .0850837    .0958143
  experience |   .0353797   .0025122    14.08   0.000     .0304544     .040305
        exp2 |  -.0465059   .0050271    -9.25   0.000    -.0563617   -.03665

In [30]:
%%stata -d cps09mr -force
regress wage education experience exp2 if mbnt==1, vce(robust)


Linear regression                               Number of obs     =      4,230
                                                F(3, 4226)        =     372.39
                                                Prob > F          =     0.0000
                                                R-squared         =     0.2334
                                                Root MSE          =      .5739

------------------------------------------------------------------------------
             |               Robust
        wage | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
   education |    .090449   .0029165    31.01   0.000     .0847312    .0961667
  experience |   .0353797   .0025854    13.68   0.000     .0303109    .0404484
        exp2 |  -.0465059   .0053069    -8.76   0.000    -.0569103   -.0361016
       _cons |   1.185209   .0461003    25.71   0.000     1.094829     1.27559
--------------

In [31]:
%stata matrix list e(b)


e(b)[1,4]
     education  experience        exp2       _cons
y1   .09044896   .03537968  -.04650594   1.1852095


In [32]:
%stata matrix list e(V)


symmetric e(V)[4,4]
             education  experience        exp2       _cons
 education   8.506e-06
experience   5.843e-07   6.684e-06
      exp2   2.997e-07  -.00001309   .00002816
     _cons  -.00010973  -.00006909   .00010557   .00212523



(b) Let $\theta$ be the ratio of the return to one year of education to the return to one year of experience for experience $=10$. Write $\theta$ as a function of the regression coefficients and variables. Compute $\hat{\theta}$ from the estimated model. [8 points]

### b) Write and compute $\theta$
donde:
$$
\theta_1 = 100*\left. \frac{\partial \text{wage}}{\partial \text{experience}} \right|_{\text{experience=10}}
\theta_2 = 00*\left. \frac{\partial \text{wage}}{\partial \text{education}} \right|_{\text{experience=10}}
\theta_3 = \theta_1 / \theta_2
$$

Al derivar la variable dependiente log(wage) con respecto a las variables education y experience, obtenemos los coeficientes de la regresión:

$\theta_1 = 100*\beta_1$

$\theta_2 = 100*\beta_2+20*\beta_3$

$\theta_3 = \frac{100*\beta_1}{100*\beta_2+20*\beta_3}$

In [33]:
%%stata -d cps09mr -force -eret dt
regress wage education experience exp2 if mbnt==1, vce(robust)


Linear regression                               Number of obs     =      4,230
                                                F(3, 4226)        =     372.39
                                                Prob > F          =     0.0000
                                                R-squared         =     0.2334
                                                Root MSE          =      .5739

------------------------------------------------------------------------------
             |               Robust
        wage | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
   education |    .090449   .0029165    31.01   0.000     .0847312    .0961667
  experience |   .0353797   .0025854    13.68   0.000     .0303109    .0404484
        exp2 |  -.0465059   .0053069    -8.76   0.000    -.0569103   -.0361016
       _cons |   1.185209   .0461003    25.71   0.000     1.094829     1.27559
--------------

In [34]:
%%stata 
margins, dydx(experience exp2) at(experience=10)


Average marginal effects                                 Number of obs = 4,230
Model VCE: Robust

Expression: Linear prediction, predict()
dy/dx wrt:  experience exp2
At: experience = 10

------------------------------------------------------------------------------
             |            Delta-method
             |      dy/dx   std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
  experience |   .0353797   .0025854    13.68   0.000     .0303109    .0404484
        exp2 |  -.0465059   .0053069    -8.76   0.000    -.0569103   -.0361016
------------------------------------------------------------------------------


In [35]:
## Coeficientes de beta
education = dt ['e(b)'][0][0]
experience = dt ['e(b)'][0][1]
experience2 = dt  ['e(b)'][0][2]


## Funciones de theta_hat = (educ_return, exp_return, theta_3)
educ_return= education*100  
exp_return= experience*100 +20*experience2
theta_3 =  np.divide(educ_return,exp_return)

In [36]:
print( "Componentes de teta",
       str(educ_return)     ,
       str(exp_return)      ,
       str(theta_3)         )

Componentes de teta 9.044895950734839 2.607849257242599 3.4683354206977555


(c) Compute $s(\widehat{\theta})$ from the estimated model. [30 points]

$\hat{\theta} = \texttt{r}\texttt{(}\hat{\beta}\texttt{)}$ 

Para hallar el error estándar asintótico de $\hat{\theta}$

$\texttt{s(}\hat{\theta}\texttt{)}$ = $\sqrt{\widehat{\text{R'}}\widehat{\text{V}}_\hat{\beta}\widehat{\text{R}}}$

donde:

$ \widehat{\text{R}} = \frac{\partial \text{r(}\hat{\beta}\text{)'}}{\partial \beta}$

$ \widehat{\text{V}}_\hat{\beta} $ es la matriz de varianza y covarianza estimada de $ \hat{\beta} $

donde:
$$
\begin{aligned}
R=\left(\begin{array}{ccc}100 & 0 & \frac{100}{100 \beta_2 + 20 \beta_3} \\ 0 & 100 & \frac{-25 \beta_1}{\left(5 \beta_2+\beta_3\right)^2} \\ 0 & 20 & \frac{-5 \beta_1}{\left(\beta_3+5 \beta_2\right)^2}\end{array}\right) = \left(\begin{array}{ccc} R_1 & R_2 & R_3\end{array}\right)
\end{aligned} 
    $$

We assume heteroskedasticity, based on Breusch Pragan

In [37]:
V = dt['e(V)']
print(V)

[[ 8.50570164e-06  5.84306166e-07  2.99667759e-07 -1.09728558e-04]
 [ 5.84306166e-07  6.68430877e-06 -1.30931653e-05 -6.90879944e-05]
 [ 2.99667759e-07 -1.30931653e-05  2.81633812e-05  1.05566736e-04]
 [-1.09728558e-04 -6.90879944e-05  1.05566736e-04  2.12523381e-03]]


In [38]:
R_transpuesta = np.array([ [100, 0, 0, 0], [0, 100, 20, 0], 
                          [100/(100*experience+20*experience2), -25*education/(5*experience+experience2)**2, 
                           -5*education/(experience2+5*experience)**2, 0]])
R = R_transpuesta.T

In [39]:
print('R','=',"\n",R_transpuesta.T)

R = 
 [[ 100.            0.           38.34577467]
 [   0.          100.         -132.99600853]
 [   0.           20.          -26.59920171]
 [   0.            0.            0.        ]]


In [40]:
print('R transpuesta','=',"\n", R)

R transpuesta = 
 [[ 100.            0.           38.34577467]
 [   0.          100.         -132.99600853]
 [   0.           20.          -26.59920171]
 [   0.            0.            0.        ]]


In [41]:
s_t = pd.DataFrame(np.diag(R.T @ V @ R ), index=[ "s2_return_educ", "s2_return_exp", "s2_theta3"], columns=["s(theta_hat)**2"])
s_t

s(theta_hat)**2
s2_return_educ         0.085057
s2_return_exp          0.025736
s2_theta3              0.051457

In [42]:
# Para hallar el error estandar asintótico
ase_t = pd.DataFrame(np.sqrt(np.diag(R.T @ V @ R)), index=[ "s_return_educ", "s_return_exp", "s_theta3"], columns=["s(theta_hat)"])
ase_t

s(theta_hat)
s_return_educ      0.291645
s_return_exp       0.160424
s_theta3           0.226841